In [1]:
import os
import numpy as np
import nibabel as nib
from nilearn import plotting as nlp
from matplotlib import pyplot as plt

/home/surchs/Venv/py35/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%matplotlib inline

In [8]:
# Paths
temp_sub = '/data1/guilimin/HCP/HCP_raw_data/100307/MNINonLinear/'
parc_name = os.path.join(temp_sub, 'wmparc.nii.gz')
rest_name = os.path.join(temp_sub, 'Results/rfMRI_REST1_LR/rfMRI_REST1_LR.nii.gz')
gm_out = os.path.join(temp_sub, 'gm_mask.nii.gz')
wm_out = os.path.join(temp_sub, 'wm_mask.nii.gz')
csf_out = os.path.join(temp_sub, 'csf_mask.nii.gz')

In [4]:
# Mask stuff
# CSF stuff
all_csf = np.array([4, 5, 14, 15, 43, 44, 72, 75, 76, 213, 221, 24])

# WM stuff
wm_hypo = np.arange(77,83)
wm_baby = np.arange(159,163)
wm_left = np.append(np.arange(3000, 3036), np.arange(3100, 3182))
wm_right = np.append(np.arange(4000, 4036), np.arange(4100, 4182))
wm_vals = np.array([2, 7, 41, 46, 85, 86, 219, 223, 703, 5001, 5002, 100, 109])

all_wm = np.concatenate((wm_baby, wm_left, wm_right, wm_vals))

# GM stuff
brain = np.concatenate((np.array([3, 42, 220]), 
                        np.arange(1000, 1036), np.arange(2000, 2036), 
                        np.arange(1100, 1181), np.arange(2100, 2181),
                        np.arange(2200, 2212)))
cerebellum = np.array([8, 47])

thalamus = np.array([9, 10, 48, 49])
caudate_putamen = np.array([11, 12, 50, 51, 136, 137])
pallidum = np.array([13, 52])
hippocampus = np.array([17, 53, 193, 197, 215, 505, 555, 1016, 2016, 1119, 2119])
amygdala = np.array([18, 54, 96, 97, 218])
insula = np.array([19, 55, 1035, 2035, 1112, 1113, 1146, 1149, 1150, 1151, 2112, 2113, 2146, 2149, 2150, 2151, 20, 56])
basal_ganglia = np.sort(np.concatenate((thalamus, caudate_putamen, pallidum, hippocampus, amygdala, insula)))
other_brain = np.concatenate((np.arange(26, 40), np.arange(58,71)))

all_brain = np.concatenate((brain, cerebellum, basal_ganglia, other_brain))

In [5]:
def make_mask(reference, values):
    mask = np.zeros_like(reference, dtype=bool)
    for val in values:
        mask += reference == val
    
    return mask

In [11]:
# Make mask
m_img = nib.load(parc_name)
mask = m_img.get_data().astype(int)

# Make WM mask
wm_mask = make_mask(mask, all_wm)
# Make GM mask
gm_mask = make_mask(mask, all_brain)
# Make CSF mask
csf_mask = make_mask(mask, all_csf)

In [12]:
# Turn those things into images
wm_img = nib.Nifti1Image(wm_mask, affine=m_img.affine, header=m_img.header)
gm_img = nib.Nifti1Image(gm_mask, affine=m_img.affine, header=m_img.header)
csf_img = nib.Nifti1Image(csf_mask, affine=m_img.affine, header=m_img.header)

In [13]:
# Save them
nib.save(wm_img, wm_out)
nib.save(gm_img, gm_out)
nib.save(csf_img, csf_out)